In [ ]:
# import libraries
import pandas as pd
import torch
import torch.nn as neuralnet
import torch.nn.functional as func
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import transforms

# tensorboard
from torch.utils.tensorboard import SummaryWriter

import os
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from torch.optim import Adam, SGD
from torch.autograd import Variable
from sklearn.metrics import accuracy_score

from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

In [ ]:
# check for gpu availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
# path where dataset is found
path = "/content/drive/MyDrive/predicting music genres using cnn/Data/"

train_data_path = path + "images_original/"
test_data_path = path + "images_original_test/"

# resize image and convert the number of channels to 1 
dataset = ImageFolder(train_data_path, transforms.Compose([transforms.Resize((30, 40)), 
                                                           transforms.ToTensor(), 
                                                           transforms.Grayscale(num_output_channels = 1)]))

test_dataset = ImageFolder(test_data_path, transforms.Compose([transforms.Resize((30, 40)), 
                                                               transforms.ToTensor(), 
                                                               transforms.Grayscale(num_output_channels = 1)]))


In [ ]:
# split dataset into batches to limit the amount of memory usage
batch_size = 10
val_size = 40
train_size = len(dataset) - val_size

train_data, val_data = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_data, batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size)
test_dataloader = DataLoader(test_dataset)

examples = iter(train_dataloader)
example_data, example_targets = examples.next()

In [ ]:
def find_mean(input_array):
  '''This function is used to find the mean of elements in an array'''
  sum = 0
  for i in input_array:
    sum = sum + i
  mean = sum/(len(input_array))
  return mean

def accuracy(outputs, lables):
  '''This function computes the accuracy of the model on teh test set'''
  _, preds = torch.max(outputs, dim = 1)
  return torch.tensor(torch.sum(preds == labels).items() / len(preds))

In [ ]:
# Neural Network models
# Baseline model - FeedForward Neural Network (FNN)
class FeedForwarNet(neuralnet.Module):
  def __init__(self, input_size, hidden_size, classes):
    super(FeedForwardNet, self).__init__()
    self.input_size = input_size
    self.layer1 = neuralnet.Linear(input_size, hidden_size) # input size 30*40
    self.relu = neuralnet.ReLU()
    self.layer2 = neuralnet.Linear(hidden_size, classes)

    def forward(self, x):
      out = self.layer1(x)
      out = self.relu(out)
      out = self.layer2(out)
      return out

# Main model - Convolution Neural Network (CNN)

class ConvNeuralNet(neuralnet.Module):
  def __init__(self):
    super(ConvNeuralNet, self).__init__()

    self.conv1 = neuralnet.Conv2d(1, 60, kernel_size = 3) # 60
    self.pool = neuralnet.MaxPool2d(2, 2)
    self.conv2 = neuralnet.Conv2d(60, 80, kernel_size = 3)
    self.conv3 = neurlanet.Conv2d(80, 150, kernel_size = 3)

    self.fc1 = neuralnet.Linear(900, 450)
    self.fc2 = neuralnet.Linear(450, 150)
    self.fc3 = neuralnet.Linear(150, 10)

    self.dropout = neuralnet.Dropout(0)

    def forward(self, x):
      x = self.pool(func.relu(self.conv1(x)))
      x = self.pool(func.relu(self.conv2(x)))
      x = self.pool(func.relu(self.conv3(x)))

      x = torch.flatten(x, 1)
      x = self.fc1(x)
      x = func.relu(x)
      x = self.fc2(x)
      x = func.relu(x)
      x = self.dropout(x)
      x = self.fc3(x)

      return x

In [ ]:
# Model hyperparameters
# Feed Forward Neural Network input and hidden size
input_size = 1200 # 30 * 40
hidden_size = 1000 

classes = 10

learning_rate = 0.001
weight_decay_value = 0
num_epochs = 50
opt_func = Adam
loss_func = neuralnet.CrossEntropyLoss()


In [ ]:
training_losses = []
validation_losses = []
accuracy_score_out = []

# define model
baseline_model = FeedForwardNet(input_size, hidden_size, classes).to(device)
model = ConvNeuralNet().to(device)

# tensor board
writer.add_graph(baseline_model, example_data.reshape(-1, 30*40))
writer.close()

writer.add_graph(model, example_data)
writer.close()


In [ ]:
def validate_model(model, model_type):
  val_losses = []
  accuracy_scores = []

  for batch in val_dataloader:
    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)

    if model_type == "FNN":
      images = images.reshape(-1, 30*40)

    output = model(images)

    val_loss = loss_func(output, labels)
    accuracy_ = accuracy(output, labels)

    val_losses.append(val_loss)
    accuracy_scores.append(accuracy_)

  val_loss_out = find_mean(val_losses)
  val_losses.clear()
  validation_losses_out.append(accuracy_score_out)

  print('Epoch : ', epoch + 1, '\t', 'loss :', val_loss_out)

  def train_model(model, model_type):

    optimizer = Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay_value)

    model.train()

    train_losses = []

    for batch in train_dataloader:
      images, labels = batch

      images = images.to(device)
      labels = labels.to(device)

      if model_type == "FNN":
        images = images.reshape(-1, 30*40)

        #forward pass
        outputs = model(images)
        train_loss = loss_func(outputs, lables)

        train_losses.append(train_loss)

        # backward and optimize
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    train_loss_out = find_mean(train_losses)
    train_losses.clear()
    trainign_losses.append(train_loss_out)
    print("train_loss", train_loss_out)

    validate_model(model, model_type)

In [ ]:
# Train Feed Forward Nerual Model
for epoch in range(num_epochs):
  train_model(baseline_model, "FNN")

plt.plot(training_losses, label = 'training losses')
plt.plot(validation_losses, label = 'validation loss')

training_losses.clear()
validation_losses.clear()

plt.plot(accurary_scores_out, label = 'accuracy_score')
accuracy_scores_out.clear()

def test_model(model, model_type):
  with torch.no_grad():
    test_losses = []
    accuracy_scores = []

    test_loss_out = 0

    accuracy_score_out = 0 

    total = 0
    correct = 0

    for data in test_dataloader:
      images, labels = data
      
      images, labels = data

      images = images.to(device)
      labels = labels.to(device)

      if model_type == "FNN":
        images = images.reshape(-1, 30*40)

      output = model(images)

      _, predicted = torch.max(output.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

      print('Accuracy of the network on test images: %d %%' % (
          100 * correct / total))
      
 test_model(baseline_model, "FNN")
 test_model(model, "CNN")

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
